In [ ]:
import pandas as pd
import pickle
DATA_PATH = "../eval/" # Adjust this to your relative path to the datasets
EXAMPLE_PATH = DATA_PATH + "examples/"
def load_pkl(path):
    with open(path, "rb") as file:
        return pickle.load(file)

In [ ]:
from datasets.arrow_dataset import Dataset

In [ ]:
def split_by_model(
    df, split_sizes: list[float] = [0.2, 0.1], random_state=4
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    df["id"] = df["model_id"].astype(str) + "_" + df["revision_id"].astype(str)
    # model_ids = df["id"].unique()
    # train_val_ids, test_ids = train_test_split(
    #     model_ids, test_size=split_sizes[-1], random_state=random_state
    # )
    # train_ids, val_ids = train_test_split(
    #     train_val_ids, test_size=split_sizes[-2], random_state=random_state
    # )
    df["num_unique_activities"] = df["unique_activities"].apply(len)
    df = df[df["num_unique_activities"] > 1]
    with open(
        DATA_PATH + "eval/train_val_test.pkl", "rb"
    ) as file:
        train_ids, val_ids, test_ids = pickle.load(file)
    train_df = df[df["id"].isin(train_ids)]
    val_df = df[df["id"].isin(val_ids)]
    test_df = df[df["id"].isin(test_ids)]
    return train_df, val_df, test_df

In [ ]:
def remove_duplicates(pair_df):
    columns = ["revision_id", "model_id", "unique_activities", "labels"]
    if "trace" in pair_df.columns:
        columns.append("trace")
    if "eventually_follows" in pair_df.columns:
        columns.append("eventually_follows")
    if "prefix" in pair_df.columns:
        columns.append("prefix")
    pair_df = pair_df.drop_duplicates(subset=columns)
    return pair_df


In [ ]:
#T_SAD_df = pd.read_pickle(DATA_PATH + "eval/eval_train_data_traces_balanced.pkl")
T_SAD_df: pd.DataFrame = load_pkl(DATA_PATH + "eval/eval_train_data_traces_balanced.pkl")
A_SAD_df = pd.read_pickle(DATA_PATH + "eval/eval_train_data_pairs_balanced.pkl")
S_NAP_df = pd.read_pickle(DATA_PATH + "eval/eval_train_prefix_data.pkl")

In [ ]:
def setify(x: str):
    set_: set[str] = eval(x)
    assert isinstance(set_, set), f"Conversion failed for {x}"
    return set_

In [ ]:
T_SAD_df["labels"] = ~(T_SAD_df.apply(lambda x: len(x["label"]) > 0, axis=1))
T_SAD_df.trace = T_SAD_df.trace.apply(lambda x: tuple(x))
T_SAD_df = remove_duplicates(T_SAD_df)
T_SAD_df.trace = T_SAD_df.trace.apply(lambda x: tuple(x))
T_SAD_df.unique_activities = T_SAD_df.unique_activities.apply(setify)
columns = ["model_id", "revision_id", "unique_activities", "trace", "labels"]
T_SAD_df = T_SAD_df.loc[:, columns]
train, val, test = split_by_model(T_SAD_df)

In [ ]:
test = test.reset_index()

In [ ]:
T_SAD_examples_llama = pd.read_csv(EXAMPLE_PATH + "llama/dataset=test_trace_epoch=1.csv")
T_SAD_examples_roberta = pd.read_csv(EXAMPLE_PATH + "roberta/dataset=test_trace_epoch=10.csv")

In [ ]:
T_SAD_examples_llama.rename(columns={'prediction': 'pred_llama', 'labels':'labels_llama'}, inplace=True)
T_SAD_examples_roberta.rename(columns={'prediction': 'pred_roberta', 'labels':'labels_roberta'}, inplace=True)

In [ ]:
merged_df = T_SAD_examples_llama.merge(T_SAD_examples_roberta, on='ids', how='inner')
filtered_df = merged_df[(merged_df["pred_llama"]!=merged_df["labels_llama"]) & (merged_df["pred_roberta"]!=merged_df["labels_llama"])]
indices = list(filtered_df.index.values)

In [ ]:
filtered_test_df = test.iloc[indices]

In [ ]:
filtered_test_df

In [ ]:
filtered_test_df.to_csv(EXAMPLE_PATH+"trace_llama_wrong_roberta_wrong.csv")